In [ ]:
import talk.config as con

# environment variable for MOSEK
con.config_mosek()
con.config_configManager()

Leveraged Portfolios
---------------------
https://en.wikipedia.org/wiki/130%E2%80%9330_fund

#### Thomas Schmelzer


### A 130/30 Equity Portfolio

- Allocate capital $C=1$. Sell short at most $c = 0.3$ to finance a long position of $1 + c$. 
- Universe of $n$ assets:
\begin{align}\mathbf{x}^{*}=\arg\max_{\mathbf{x} \in \mathbb{R}^n}& \mu^{T}\mathbf{x}\\
\text{s.t. } &\Sigma\,x_i=1\\
             &\Sigma\,\lvert x_i\rvert \leq 1 + 2c\\
            &\sqrt{\mathbf{x}^T\mathbf{C}\mathbf{x}} \leq \sigma_{\max}
\end{align}


### Cholesky decomposition of the covariance matrix:
$$\sqrt{\mathbf{x}^T\mathbf{C}\mathbf{x}}=\sqrt{\mathbf{x}^T\mathbf{G G}^T\mathbf{x}}=\rVert{\mathbf{G}^T\mathbf{x}}\lVert_2$$
Introduce the cone $[y, G^T x] \in \mathcal{Q}_{n+1}$.


### The absolute value:
$$\lvert{x_i}\rvert=\sqrt{x_i^2}=\rVert{x_i}\lVert_2$$
The sum of absolute values (e.g. the $1$-norm) is replaced by $n$ cones of dimension $2$, e.g.
$[t_i, x_i] \in \mathcal{Q}_2$. Now
$$\Sigma\,t_i \leq 1 + 2c$$ implies $$\Sigma\,\lvert x_i\rvert \leq 1 + 2c$$

\begin{align}\mathbf{x}^{*}=\arg\max_{\mathbf{x} \in \mathbb{R}^n}& \mu^{T}\mathbf{x}\\
\text{s.t. } &\Sigma\,x_i=1\\
             &\Sigma\, t_i \leq 1 + 2c\\
             &[y, G^T x] \in \mathcal{Q}_{n+1}\\
             &y \leq \sigma_{\max}\\
             &[t_i, x_i] \in \mathcal{Q}_2,\,\,i \in [1,\ldots,n]
\end{align}


In [ ]:
from mosek.fusion import Model, DenseMatrix, Domain, Expr, ObjectiveSense
import numpy as np
# make some random data, e.g. cov-matrix and expected returns
n = 100
c = 0.9
C = c * np.ones((n, n)) + (1 - c) * np.eye(n)
mu = 0.05 * np.sin(range(0, n))
# maximal volatility and leverage...
sigma_max = 1.0
excess = 0.3

with Model('equity') as model:
    w = model.variable("w", mu.size, Domain.unbounded())
    t = model.variable("t", mu.size, Domain.unbounded())

    # e'*w = 1
    model.constraint(Expr.sum(w), Domain.equalsTo(1.0))

    # e'*t <= 1 + 2*excess
    model.constraint(Expr.sum(t), Domain.lessThan(1.0 + 2 * excess))

    # attach the cones t_i >= Norm(w_i)
    model.constraint(Expr.hstack(t, w), Domain.inQCone(int(w.size()), 2))

    # Norm(G^T*w) <= sigma_max 
    G_transpose = np.transpose(np.linalg.cholesky(C))
    model.constraint(Expr.vstack(Expr.constTerm(sigma_max), Expr.mul(DenseMatrix(G_transpose), w)), Domain.inQCone())

    # maximize objective mu'w
    model.objective(ObjectiveSense.Maximize, Expr.dot(mu, w)), model.solve()
    f = np.array(w.level())

print("Sum of positive weights: {0}".format(np.sum(f[f > 0])))
print("Sum of negative weights: {0}".format(np.sum(f[f < 0])))
print("Sum of all weights:      {0}".format(np.sum(f)))

### Summary

- Leverage is here a constraint for the $1$-norm of the weight vector.


- We replace the $1$-norm of a vector of length $n$ with $n$ cones each of dimension $2$.


- We compute the Cholesky decomposition of a $n \times n$ covariance matrix to introduce a cone of dimension $n + 1$.

<a href=http://localhost:8888>Back to Overview</a>

In [ ]:
from mosek.fusion import Model, DenseMatrix, Domain, Expr, ObjectiveSense
import numpy as np
# make some random data, e.g. cov-matrix and expected returns
n = 100
c = 0.9
C = c * np.ones((n, n)) + (1 - c) * np.eye(n)
mu = 0.05 * np.sin(range(0, n))
# maximal volatility and leverage...
sigma_max = 1.0
excess = 0.3

def one_norm(model, v):
    t = model.variable(int(v.size()), Domain.greaterThan(0.0))
    model.constraint(Expr.hstack(t, v), Domain.inQCone(int(v.size()), 2))
    return Expr.sum(t)

with Model('equity') as model:
    w = model.variable("w", mu.size, Domain.unbounded())
    
    # e'*w = 1
    model.constraint(Expr.sum(w), Domain.equalsTo(1.0))

    # e'*t <= 1 + 2*excess
    model.constraint(one_norm(model, w), Domain.lessThan(1.0 + 2 * excess))

    # Norm(G^T*w) <= sigma_max 
    G_transpose = np.transpose(np.linalg.cholesky(C))
    model.constraint(Expr.vstack(Expr.constTerm(sigma_max), Expr.mul(DenseMatrix(G_transpose), w)), Domain.inQCone())

    # maximize objective mu'w
    model.objective(ObjectiveSense.Maximize, Expr.dot(mu, w)), model.solve()
    f = np.array(w.level())

print("Sum of positive weights: {0}".format(np.sum(f[f > 0])))
print("Sum of negative weights: {0}".format(np.sum(f[f < 0])))
print("Sum of all weights:      {0}".format(np.sum(f)))